# Practical work on SAR statistics - IMA207

### Emanuele DALSASSO, Florence TUPIN

Images of the practical work can be found on: 
https://perso.telecom-paristech.fr/dalsasso/TPSAR/
and 
https://perso.telecom-paristech.fr/tupin/TPSAR/DATA/images/

Some useful functions are available in the file *mvalab.py*. 

### Study of homogeneous areas

You have at your disposal a set of images coming from different sensors and with different characteristics on the same area of Flevoland in Netherlands (for each sensor and acquisition mode, an homogeneous area of sea has been selected with *mer* extension, and an area of farmland with  *centre* extension):
- Sentinel-1 sensor (ESA), SLC (Single look Complex) data and GRD (Ground Range Detected) data ;
- ERS sensor (ESA), PRI product (ground range data);
- Alos sensor (JAXA), SLC (Single look Complex) data.



### First steps
The first two cells will set the display at full screen and disable cell scrolling, allowing for a more pleasant display of the figures
  

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
#%matplotlib notebook
# if using google colab:
%matplotlib inline 
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import signal
import mvalab as mvalab

plt.rcParams['figure.figsize'] = [8, 8]
plt.rcParams['figure.max_open_warning'] = 30

## A. Single look data distributions 
In this part, we will use an SLC (Single Look Complex) image and analyze its distribution. 
The image has been acquired by the Sentinel-1 sensor over the Lelystad zone (very flat area with fields crops). 
Vizualize the amplitude image and interpret it. 

NB amplitude image is given by the modulus of the electro-magnetic field and intensity is the square of the amplitude (proportional to the signal power). 

In [ ]:
import mvalab as mva
pageweb="https://perso.telecom-paristech.fr/dalsasso/TPSAR/pilelely/"
image='Lely.CXF'
im_slc_senti_lely_liste=mvalab.imz2mat(pageweb+image);
im_slc_senti_lely = im_slc_senti_lely_liste[0]
ncol=im_slc_senti_lely_liste[1]
nlig=im_slc_senti_lely_liste[2]

plt.rcParams['figure.figsize'] = [15, 15]
# plot the amplitude image
mva.visusar(im_slc_senti_lely)
#- insert code here to display the image
# because the image is big you can display a 1024 x 1024 pixels part 

### A.1. Data distributions for an homogeneous area
Select a **physically homogeneous** area and compute the distribution of the real part, imaginary part, phase, intensity and amplitude. Some useful functions are:
- `np.angle`
- `np.real`
- `np.imag` 

Compute the histograms for these variables and the corfficient of variation ($\gamma=\frac{\sigma}{\mu}$) in intensity and amplitude. 

NB The Goodman model is valid only on homogeneous area. That is why it is important to select pixels sharing the same distribution (with the same underlying reflectivity).


In [ ]:
# Select a crop of the image (around 200 by 200 pixels)
crop_slc = im_slc_senti_lely[900:1000,0:200]
mvalab.visusar(crop_slc)

# Compute amplitude, intensity, phase, real and imaginary part
amp_senti_lely = np.abs(crop_slc)
int_senti_lely = np.square(amp_senti_lely)
ph_senti_lely = np.angle(crop_slc)
real_senti_lely = np.real(crop_slc)
imag_senti_lely = np.imag(crop_slc)

In [ ]:
# Plot the histograms and verify they match the theoretical distribution
plt.rcParams['figure.figsize'] = [8, 8]

plt.figure()
plt.hist(amp_senti_lely.ravel(),bins='auto',density=True,range=[0.,100])  
plt.title('histogram of amplitude')
plt.show()  

plt.figure()
#compute histogram for intensity data 
plt.hist(int_senti_lely.ravel(),bins='auto',density=True,range=[0.,5000])
plt.title('histogram of intensity')
plt.show()

plt.figure()
#compute histogram for phase data 
plt.hist(ph_senti_lely.ravel(),bins='auto',density=True,range=[-np.pi,np.pi])
plt.title('histogram of phase')
plt.show()

plt.figure()
#compute histogram for real part 
plt.hist(real_senti_lely.ravel(),bins='auto',density=True,range=[-100,100])
plt.title('histogram of real part')
plt.show()

plt.figure()
#compute histogram for imaginary part
plt.hist(imag_senti_lely.ravel(),bins='auto',density=True,range=[-100,100])
plt.title('histogram of imaginary part')
plt.show()

In [ ]:
# Compute the coefficient of variation on the homogeneous crop in amplitude format

m_A = np.mean(amp_senti_lely)
sigma_A = np.std(amp_senti_lely)
coeff_var_A = sigma_A/m_A
print("Variance Amplitude " + str(coeff_var_A))

# Compute the coefficient of variation on the homogeneous crop in intensity format
m_A = np.mean(int_senti_lely)
sigma_A = np.std(int_senti_lely)
coeff_var_A = sigma_A/m_A
print("Variance Intensité " + str(coeff_var_A))

## Question 1
What are the distributions followed by real part, imaginary part, phase, intensity and amplitude on an homogeneous area ? 
What are the values for the coefficient of variation for amplitude and intensity data? Are they in accordance with the theoretical values ? 

---
__Réponse:__

L'amplitude suit une loi de Rayleigh. 
L'intensité suit une loi exponentielle décroissante.
La phase suit une loi uniforme sur [-pi,pi].
Les parties réelles et imaginaires suivent des lois gaussiennes. 

Les coefficients de variations sont 0.54 pour l'amplitude et 1.05 pour l'intensité, très proches des valeurs théoriques (0.523 et 1). LA différence peut s'expliquer car les hypothès ne sont pas totalement respectées.

## B. Multi-looking of data
A common way to reduce the speckle is to multi-look the data.

Compute a multi-look down-sampled image of Lely SLC data `im_slc_senti_lely` using a factor of 1 in the vertical direction and 4 in horizontal (this will give almost square pixels). You can do so by convolving the image with an average window with the aforementioned dimensions. Use `signal.convolve2d` with `mode = 'same'`.



In [ ]:
# Create the mask that will be used to average pixels within the horizontal window of dimension 4
masque_vert = np.ones((1,4))/4

# Compute multilooking in intensity by convolving the entire image with the window just created
sentinel_ml_int =signal.convolve2d(np.multiply(im_slc_senti_lely,np.conj(im_slc_senti_lely)),masque_vert,mode='same')

# Downsample the image by a factor of 4, so to have squared pixels, and plot it in amplitude 
mvalab.visusar(np.sqrt(sentinel_ml_int[:,::4]))
plt.suptitle(u'SENTINEL : Multivue en intensité')

In [ ]:
# Compute multilooking in amplitude
sentinel_ml_amp = signal.convolve2d(np.abs(im_slc_senti_lely),masque_vert,mode='same')

# Downsample the image by a factor of 4, so to have squared pixels, and plot it in amplitude 
mvalab.visusar(sentinel_ml_amp[:,::4]) 
plt.suptitle(u'SENTINEL : Multivue en amplitude')

In [ ]:
# Compute the ratio between the multilooked intensity and amplitude to see the differences.
# WARNING: prevent division by 0
ratio = np.divide(np.abs(im_slc_senti_lely[:,::4]),np.sqrt(sentinel_ml_int[:,::4])+0.000001)
ratio = np.abs(ratio).astype(np.float32)
mvalab.visusar(ratio,0)

## Question 2.
Comment the effect of multi-looking. 

Multilooking can be computed both in intensity and in amplitude format. Can you see the differences?

---
__Résultats :__
    Le multivue réduit la taille de l'image dans la direction horizontale, mais les pixels sont carrés. On observe moins de bruit, et les pixels sont isotropes ce qui rend l'image plus facilement interprétable.

Les résultats se ressemblent beaucoup, on voit que la moyenne du ratio est environ de 1 et que l'écart type est très faible. 

Sur l'image du ratio on observe cependant quelques différences. La moyenne est autour de 1 avec de faibles variations, mais autour des points brillants, on observe des petites anomalies. Cela montre que les techniques de moyennage n'ont pas exactement le même effet autour des points très lumineux.

## C. Computation of the Equivalent Number of looks on homogeneous areas
In this part you have at your disposal 3 images of a part of the sea. One is a Sentinel-1 GRD image and the other one is an ERS image. The multi-looking has been done by the data provider (ESA, European Space Agency).
Use the value of the coefficient of variation to find the equivalent number of looks (ENL) of the Sentinel-1 GRD and ERS data ($\gamma_I=\frac{1}{\sqrt{L}}$ for intensity data, $\gamma_A=\frac{0.523}{\sqrt{L}}$ for amplitude). Normally the ENL should be an integer corresponding to the number of aberaged samples. 



In [ ]:
#Intensité
pageweb='https://perso.telecom-paristech.fr/tupin/TPSAR/DATA/images/'
image = 'SentinelGRD_flevoland_mer.imw'
im_sentigrd_mer = mvalab.imz2mat(pageweb+image)
mvalab.visusar(np.abs(im_sentigrd_mer[0]))


# compute coefficient of variation and number of looks in amplitude
amp_grd=np.abs(im_sentigrd_mer[0])
m_grd= np.mean(amp_grd)
sigma_grd = np.std(amp_grd)
coeff_var_grd = sigma_grd/m_grd
L_grd = np.square(0.523/coeff_var_grd)
print('--- coeff var and L ---')
print(coeff_var_grd)
print("L calculé en amplitude " + str(L_grd))

In [ ]:
int_grd=amp_grd**2
var_grd = np.std(int_grd.ravel())/np.mean(int_grd.ravel())
L_grd = (1/var_grd)**2
print('--- coeff var and L ---')
print(coeff_var_grd)
print("L calculé en intensité",L_grd)

In [ ]:
image = 'ERS_flevoland_mer.imw'
im_ers_mer = mvalab.imz2mat(pageweb+image)
mvalab.visusar(np.abs(im_ers_mer[0]))

# compute coefficient of variation and number of looks in amplitude
amp_ers=np.abs(im_ers_mer[0])
m_ers= np.mean(amp_ers)
sigma_ers = np.std(amp_ers)
coeff_var_ers = sigma_ers/m_ers
L_ers = np.square(0.523/coeff_var_ers)

print('--- coeff var and L ---')
print(coeff_var_ers)
print("L calculé en amplitude " + str(L_ers))

In [ ]:
int_ers=amp_ers**2
var_ers = np.std(int_ers.ravel())/np.mean(int_ers.ravel())
L_ers = (1/var_ers)**2 
print('--- coeff var and L ---')
print(coeff_var_ers)
print("L avec l'intensité",L_ers)

## Question 3.
Comment the number of looks you have found for Sentinel1-GRD and ERS data.

---
__Réponse :__

Pour Sentinel1-GRD, on obtient environ les 5 vues réelles attendu, car on trouve 4.79 en amplitude.

Pour ERS, on obtient 2.69 en amplitude, soit environ 3 vues réelles.

Si l'estimation de L est bonne, on n'obtient pas de valeurs exactes car les conditions de Goodman ne sont pas respectées.

## D. Local coefficient of variation

The coefficient of variation $\gamma=\frac{\sigma}{\mu}$ (standard deviation normalized by the mean) is an indication of the local homogeneity of the scene. 
It can be computed locally around each pixel using a moving window.

Using 2D convolution to speed up the processing, compute the images of coefficient of variation. 


In [ ]:
# Select one of the following images: 
#'SentinelSLC_flevoland_centre.cxs', 
#'Alos_flevoland_centre.cxf',
#'ERS_flevoland_centre.imw'

image = 'SentinelSLC_flevoland_centre.cxs'
ima_slc = mvalab.imz2mat(pageweb+image)

# take the intensity 
ima_int = np.multiply(ima_slc[0], np.conj(ima_slc[0]))
# crop of the image to ease display and computation
ima_int=ima_int[0:512, 1024:2048]
mvalab.visusar(np.sqrt(ima_int));


# create the average window
size_window = 7
masque_loc = np.ones((size_window,size_window))/size_window**2

# compute the image of the local means in intensity
ima_int_mean = signal.convolve2d(ima_int,masque_loc,mode='same')

# compute the image of the local variances (var{I} = E{I^2} - E{I}^2)
ima_int_square = np.multiply(ima_int,ima_int) # I^2 
ima_int_mean_square = signal.convolve2d(ima_int_square,masque_loc,mode='same')# E{I^2}
ima_variance = ima_int_mean_square - np.multiply(ima_int_mean,ima_int_mean) 

# compute the image of the local coefficient of variation
ima_coeff_var = np.divide(np.sqrt(ima_variance),ima_int_mean)
mvalab.visusar(ima_coeff_var,6)
plt.suptitle(u'Coefficient of variation')

# plot the standard deviation: comment the result
mvalab.visusar(np.sqrt(ima_variance),2)
plt.suptitle(u'Standard deviation')

## Question 4. 
Comment the results of the image of local coefficient of variation and local standard deviation. Which structures of the image are highlighted with the coefficient of variation ? What is the influence of the window size ? 
Why the local standard deviation is not adapted to measure the local homogeneity of the scene ?

---
__Réponse :__

L'image du coefficient de variation est un détecteur de contours : sa valeur est forte au niveau des cibles très brillantes et des contours entre un champ clair et un champ sombre, mais la valeur de l'image est faible au niveau des zones homogènes telles que l'intérieur des champs.

La fenêtre détermine la taille sur laquelle on va moyenner l'image, soit la taille des pixels de la nouvelle image.
Un masque trop grand floute beaucoup l'image, on perd en détail et en information. On risque de ne pas bien détecter les contours si la taille du masque est plus grande que certain champs.
Si le masque trop petit, l'image de la variance est très bruitée. Il faut prendre une fenêtre suffisament grande pour détecter les changements de couleurs.

L'écart-type local detecte bien les points lumineux, mais c'est un mauvais détecteur de l'hétérogénéité.
En effet, les champs clairs sont considérés comme hétérogènes. 

## E. Image despeckling: simple averaging and Lee filter

The local coefficient of variation is also used in a very famous filter for SAR images: the Lee filter. 
The principle of the filter is to combine the pixel value $I_s$ (intensity value of pixel $s$) and the local mean $\hat{\mu}_{s}$ depending on the local coefficient of variation $\hat{\gamma}_s$ with the following formula :
$
  \hat{I}_s= \hat{\mu}_{s}+k_s (I_s-\hat{\mu}_{s})$

and
$
  k_s=1- \frac{\gamma_{Sp}^2}{\hat{\gamma}^2_s}
$

$\gamma_{Sp}$ is the theoretical value of the coefficient of variation for a pure speckle ($\gamma_{Sp}=\frac{1}{\sqrt{L}}$ for a L-look intensity image).

Using the previous map, compute the resulting image with Lee filter. 

Warning : $k$ should be in $[0,1]$. 



In [ ]:
# compute the image of ks coefficients, by taking ima_coeff_var previously computed
ks = 1-(1/(ima_coeff_var*ima_coeff_var))

# force ks to have values comprised in the range [0,1]
ks[ks<0]=0
mvalab.visusar(ks,0)

# filter the image
A=np.multiply(ks,ima_int-ima_int_mean)
image_lee_filtered = ima_int_mean+A
mvalab.visusar(np.sqrt(image_lee_filtered))
plt.suptitle(u'Image denoised using Lee filter')
mvalab.visusar(np.sqrt(ima_int))
plt.suptitle(u'Original image')

In [ ]:
# compare it with a local mean
mvalab.visusar(np.sqrt(ima_int_mean))

## Question 5.
Comment the result and compare with a local mean. 

---
__Réponse :__

Avec le filtre de Lee, l'image est moyennée, mais il reste du bruit, là ou l'images était hétérogène. Dans ce cas on garde les valeurs bruitées du début, plutôt que de moyenner

On utilise le coefficient de cariation pour vérifier si la zone est homogène avant de moyenner. Si la zone n'est pas homogène, on n'effectue pas le moyennage

Par rapport à la moyenne locale, qui est un peu débruitée mais floue, le filtre de Lee est meilleur, car il préserve mieux les bords et les points brillants. Ces points sont en effets préservés grâce à ks.


### Filtering of image "Lely" and comparison with a deep learning algorithm
Repeat the process done above to denoise a crop of image "Lely" using the Lee filter. Then, compare it with the result of a deep learning algorithm called SAR-CNN.

In [ ]:
# REPEAT THE PROCESS WITH THE CROP FROM LELY
plt.rcParams['figure.figsize'] = [8, 8]
x = 470
y = 2230
pat_size = 512
lely_crop_slc = im_slc_senti_lely[x : x+pat_size, y : y+pat_size]
mvalab.visusar(np.abs(lely_crop_slc))

# take the intensity 
ima_int = np.multiply(lely_crop_slc, np.conj(lely_crop_slc))
mvalab.visusar(np.sqrt(ima_int));

# create the average window
size_window = 7
masque_loc = np.ones((size_window,size_window))/(size_window*size_window)

# compute the mean image
ima_int_mean = signal.convolve2d(ima_int,masque_loc,mode='same')

# compute the variance image (var{I} = E{I^2} - E{I}^2)
ima_int_square = np.multiply(ima_int,ima_int) # I^2 
ima_int_mean_square =  signal.convolve2d(ima_int_square,masque_loc,mode='same') # E{I^2}
ima_variance = ima_int_mean_square-np.multiply(ima_int_mean,ima_int_mean) 

# compute coefficient of variation
ima_coeff_var = np.divide(np.sqrt(ima_variance),ima_int_mean)
mvalab.visusar(ima_coeff_var)

In [ ]:
# compute ks, by taking ima_coeff_var previously computed
ks = 1-(1/ima_coeff_var)

# force k to have values comprised in the range [0,1]
ks[ks<0]=0
mvalab.visusar(ks)

# filter the image
image_lee_filtered = ima_int_mean+np.multiply(ks,ima_int-ima_int_mean)
mvalab.visusar(np.sqrt(image_lee_filtered))
mvalab.visusar(np.sqrt(ima_int))

### Denoised image: SAR-CNN
The Lee filter presents some limits. More recent approaches to suppress noise rely on sofisticated algorithms. You can plot the image of Lely denoised using a deep learning algorthm called SAR-CNN and compare visually the result with the image filtered using the Lee filter.
  

In [ ]:
image = 'Lely_denoised.IMF'
pageweb = 'https://perso.telecom-paristech.fr/dalsasso/TPSAR/pilelely/denoised_SARCNN/'
im_lely_multitemp_denoised = mvalab.imz2mat(pageweb+image)
mvalab.visusar(im_lely_multitemp_denoised[0])

## Question 7.
Do a comparison between the CNN filetring and the Lee filter. What is the problem for CNN methods for SAR images ? 

---

__Réponse :__

Le filtre de Lee garde du bruit, même dans les zones homogènes, et encore plus dans les zones avec beaucoup de points lumineux

Au contraire le filtre CNN lisse trop les zones homogènes, commes les champs, ce qui rend l'image trop floue. Par contre le filtre CNN filtre bien les zones avec beaucoup de rétrodiffuseurs.


## F. Method noise
Performances of a denoising algorithm can be visually interpreted by looking at the *residual noise* (i.e. the ratio between the noisy image and the denoised image, in intensity). For a quantitative evaluation, we can look at the noise statistics, knowing that, in intensity, statistics of speckle S are:
- $\mu_S=1$
- $\sigma^2_S = \frac{1}{L}$

In [ ]:
# Plot the residual noise as division between ima_int and the denoised image
# study the moments and the histogram of the residual noise

res_noise_lee = ima_int/image_lee_filtered
mvalab.visusar(res_noise_lee)
plt.hist(res_noise_lee.flatten(),density='True')
mean_lee = np.mean(res_noise_lee)
var_lee = np.var(res_noise_lee)
print('LEE FILTER: mean = '+str(mean_lee)+' and var = '+str(var_lee))

# warning: im_lely_multitemp_denoised[0] is in amplitude
res_noise_deep = ima_int/(np.abs(im_lely_multitemp_denoised[0]**2))
mvalab.visusar(res_noise_deep)
plt.hist(res_noise_deep.flatten(),density='True')
mean_deep = np.mean(res_noise_deep)
var_deep = np.var(res_noise_deep)
print('SAR-CNN: mean = '+str(mean_deep)+' and var = '+str(var_deep))

## Question 8. 
What is the interest of computing the method noise ? What are your conclusions for the two previous filters ?

---
__Réponse :__

Cette visualisation met en avant les zones qui ont été débruitées ou non. S l'image était correctement débruitée, nous obtiendrions du bruit pur. 

Sur le filtre de Lee, les zones sombres correspondent aux zones non débtruitées, là ou ks était élevé. Les contours apparaissent en sombre sur l'image, car ce sont des zones de forte variance. La moyenne est inférieure à 1, car certaines zones n'ont pas été transformées, donc le filtre est biaisé.

Pour la méthode de deep learning, le débruitage est plus homogène, mais on aperçoit enclore certains contours.
La moyenne est très proche de 1, ce qui montre que le débruitage n'est pas biaisé, mais la variance est plus grande que le résultat attendu.

On voit que les histogrammes ne correspondent pas exactement à la distribution de Rayleigh attendue.